In [9]:
## Load Dataframe
import pandas as pd
import warnings
import fnmatch
import os

pd.set_option('future.no_silent_downcasting', True)
root = "cookieScan"
def loadDataFrameFromFileRegex(root, regex):
    df_arr = []
    for path, subdirs, files in os.walk(root):
        for name in files:
            if fnmatch.fnmatch(name, regex) and os.path.getsize(os.path.join(path, name)) > 0:
                # print(os.path.join(path, name))
                df = pd.read_csv(os.path.join(path, name))
                df_arr.append(df)
    if not df_arr:
        warnings.warn("No matching file found in "+root+" for regex: "+regex+". Empty dataframe will be returned." )
        return pd.DataFrame()    
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=FutureWarning)      
        return pd.concat(df_arr, ignore_index=True)
        
df = loadDataFrameFromFileRegex(root, 'COOKIE*.csv')
display(df)

df = df.groupby(['tenant','domain', 'scan-id']).agg(first=('timestamp', 'min'),last=('timestamp', 'max'), count=('url-count', 'sum')).reset_index()
df.to_csv("cookieScan_report.csv")

,timestamp,tenant,domain,scan-id,url-count
0,1716940800000,tenant588806__savegnagocombr,https://www.paulistaoatacadista.com.br/,259,1201
1,1716940800000,tenant588806__savegnagocombr,https://www.savegnago.com.br/,257,1199
2,1716940800000,tenant588806__savegnagocombr,blog.savegnago.com.br,258,173
3,1716940800000,tenant588806__savegnagocombr,http://iaparecidosavegnago.org.br/,261,40
4,1716940800000,tenant588806__savegnagocombr,https://paulistao-atacadista.zendesk.com/hc/pt-br,260,4
...,...,...,...,...,...
8512,1717027200000,tenant859896__instacartcom,https://krogerconvenience.instacart.com,3848,5
8513,1717027200000,tenant469014__alpargatascom,alpashop.alpargatas.com.br,519,2
8514,1717027200000,tenant267705__freemancom,https://rb.gy/04l8ax,12,1
8515,1717027200000,tenant508688__kpmgcombr,https://frtb.kpmg.com.br/,783,1
